In [1]:
import pandas as pd

df = pd.read_csv("Dataset/ar_reviews_100k.tsv", sep = '\t')

label_mapping = {"Positive": 1, "Negative":-1 , "Mixed": 0}

df = df.rename(columns = {"class":"label"})

df["label"] = df["label"].map(label_mapping)

In [2]:
df["label"].value_counts()
# print(df.head())
print(df["text"])
print(df["label"])

0        ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...
1        أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...
2        هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...
3        خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...
4        ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...
                               ...                        
99994    معرفش ليه كنت عاوزة أكملها وهي مش عاجباني من ا...
99995    لا يستحق ان يكون في بوكنق لانه سيئ . لا شي. لا...
99996    كتاب ضعيف جدا ولم استمتع به. فى كل قصه سرد لحا...
99997    مملة جدا. محمد حسن علوان فنان بالكلمات، والوصف...
99998    لن ارجع إليه مرة اخرى . قربه من البحر. المكان ...
Name: text, Length: 99999, dtype: object
0        1
1        1
2        1
3        1
4        1
        ..
99994   -1
99995   -1
99996   -1
99997   -1
99998   -1
Name: label, Length: 99999, dtype: int64


In [3]:
#https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

from sklearn.model_selection import train_test_split

train,test  = train_test_split( df, train_size=0.7,test_size=0.3, random_state=101)

print("Train set: ")
print(train["label"].value_counts())
print("---------------------------")
print ("Test set: ")
print (test["label"].value_counts())

Train set: 
 1    23378
-1    23315
 0    23306
Name: label, dtype: int64
---------------------------
Test set: 
 0    10027
-1    10018
 1     9955
Name: label, dtype: int64


In [4]:

x = train['text']
y = train['label']

x_test = test['text']
y_test = test['label']

In [5]:
print("Traning data : ")
print(x)
print(y)

# print("Testing data")
# print(x_test)
# print(y_test)

Traning data : 
19755    أحببتها كثيرا. أخذتني إلى عالم آخر لقد عشتها ب...
78590                                       خامة رديئه جدا
42068    مقبول. الموقع جيد. الخدمة عادية جدا ولا تليق ب...
28438    اول كتاب تشعر انه حقيقى وصادق ومخلص عن حرب اكت...
68764    لم اكمله توقفت في منتصفه. لم اجد لذلك من متعة....
                               ...                        
5695     الكتاب الذي دفعني إلى عالم الكتابة. كنت أتمنى ...
73542    كتاب خفيف وظريف : عرفني أكثر على بعض مشاهير تويتر
83281    فندق لطيف إذا كنت تحب الصراصير لابد أن هذا الف...
83467    اعرف ان الروايه تكون من نسج خيال المؤلف، اما ه...
45919                           مرضي. لا شيء. ازعاج المرقص
Name: text, Length: 69999, dtype: object
19755    1
78590   -1
42068    0
28438    1
68764   -1
        ..
5695     1
73542   -1
83281   -1
83467   -1
45919    0
Name: label, Length: 69999, dtype: int64


In [6]:
#https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html
#https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
import numpy as np
 
def train_model(model, data, targets):
    text_clf = Pipeline([('vect',
    TfidfVectorizer(lowercase=False,ngram_range=(1, 2))),
    ('clf', model)])
    
    text_clf.fit(data, targets)
    return text_clf
def get_accuracy(trained_model,x, y):
    predicted = trained_model.predict(x)
    accuracy = np.mean(predicted == y)
    return accuracy

In [11]:
#https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html
from sklearn.naive_bayes import MultinomialNB
trained_clf_multinomial_nb = train_model(MultinomialNB(), x, y)
accuracy = get_accuracy(trained_clf_multinomial_nb,x_test, y_test)
print("Test dataset accuracy with MultinomialNB:", accuracy)


Test dataset accuracy with MultinomialNB: 0.6580333333333334


In [37]:
#https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html
from sklearn.svm import LinearSVC
trained_clf_linearSVC = train_model(LinearSVC(), x, y)
accuracy = get_accuracy(trained_clf_linearSVC,x_test, y_test)
print("Test dataset accuracy with LinearSVC:", accuracy)

Test dataset accuracy with LinearSVC: 0.6772666666666667


In [10]:
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix,accuracy_score, classification_report
pipe = make_pipeline(TfidfVectorizer(),MultinomialNB())
pipe.fit(x,y)
prediction = pipe.predict(x_test)
print(f"Accuracy score is {accuracy_score(y_test, prediction):.2f}")
print(classification_report(y_test, prediction))

Accuracy score is 0.64
              precision    recall  f1-score   support

          -1       0.67      0.72      0.69     10018
           0       0.55      0.59      0.57     10027
           1       0.72      0.62      0.66      9955

    accuracy                           0.64     30000
   macro avg       0.65      0.64      0.64     30000
weighted avg       0.65      0.64      0.64     30000

